In [1]:
import pennylane as qml
from pennylane import numpy as np

# Define the oracle for the marked element (e.g., |11111>)
def oracle(n_qubits, marked_element):
    def oracle_circuit():
        for i, bit in enumerate(reversed(marked_element)):
            if bit == '0':
                qml.PauliX(i)
        qml.MultiControlledX(control_wires=list(range(n_qubits-1)), wires=n_qubits-1)
        for i, bit in enumerate(reversed(marked_element)):
            if bit == '1':
                qml.PauliX(i)
    return oracle_circuit

# Define the diffuser (inversion about the mean)
def diffuser(n_qubits):
    def diffuser_circuit():
        for i in range(n_qubits):
            qml.Hadamard(i)
        for i in range(n_qubits):
            qml.PauliX(i)
        qml.MultiControlledX(control_wires=list(range(n_qubits-1)), wires=n_qubits-1)
        for i in range(n_qubits):
            qml.PauliX(i)
        for i in range(n_qubits):
            qml.Hadamard(i)
    return diffuser_circuit

# Grover's algorithm implementation
def grover_search(n_qubits, marked_element, num_iterations):
    dev = qml.device("default.qubit", wires=n_qubits)

    @qml.qnode(dev)
    def circuit():
        # Initialize in uniform superposition
        for i in range(n_qubits):
            qml.Hadamard(i)
        # Grover iterations
        for _ in range(num_iterations):
            oracle(n_qubits, marked_element)()
            diffuser(n_qubits)()
        return qml.probs(wires=range(n_qubits))
    
    return circuit()

# Example usage: search for |11111> in 5 qubits
n_qubits = 5
marked_element = '10000'
num_iterations = 3  # For 5 qubits, floor(pi/4*sqrt(2**5)) ≈ 3

probs = grover_search(n_qubits, marked_element, num_iterations)
print("Probabilities:", probs)
print("Most probable state:", np.argmax(probs))

Probabilities: [0.23632812 0.23098755 0.01757812 0.0161438  0.01757812 0.0161438
 0.01757812 0.0161438  0.01757812 0.0161438  0.01757812 0.0161438
 0.01757812 0.0161438  0.01757812 0.0161438  0.01757812 0.0161438
 0.01757812 0.0161438  0.01757812 0.0161438  0.01757812 0.0161438
 0.01757812 0.0161438  0.01757812 0.0161438  0.01757812 0.0161438
 0.03125    0.02932739]
Most probable state: 0


c:\Users\xillegas\.conda\envs\qiskit\lib\site-packages\pennylane\ops\op_math\controlled_ops.py:1046: UserWarning: The control_wires keyword will be removed soon. Use wires = (control_wires, target_wire) instead. See the documentation for more information.
  warnings.warn(
